# Chapter 2 : Modeling in the Frequency Domain (Part 2)
---

In [1]:
import sys

if (path := "C:/Users/Tom/pycharm-projects/python-control") not in sys.path:
    sys.path.append(path)

import sympy as sp

from python_control import Quantity, TransferFunction
from python_control.modeling.electrical import (
    Inductor, Resistor, Capacitor,
    Mesh, Circuit
)

Q_ = Quantity

## 2.4 : Electrical Network Transfer Functions

### Example 2.7 : Transfer Function - Single Loop via Transform Methods

Find the transfer function relating the capacitor voltage $V_c(s)$ to the input voltage $V(s)$.

![RLC network](.\images\example_2-6.png)

First we define the electrical components in the network and display their impedance $Z$ and admittance $Y$:

In [2]:
inductor = Inductor('L')
Z_L = inductor.Z
Y_L = inductor.Y
Z_L.expr

1.0*L*s

In [3]:
resistor = Resistor('R')
Z_R = resistor.Z
Y_R = resistor.Y
Z_R.expr

1.0*R

In [4]:
capacitor = Capacitor('C')
Z_C = capacitor.Z
Y_C = capacitor.Y
Z_C.expr

1.0/(C*s)

In fact, the impedance and the admittance of an electrical component are objects of class `TransferFunction`.

The voltage $V_c$ across the capacitor depends on the current $I$ that flows through the RLC series circuit. To find this current, we first determine the total impedance of the circuit: 

In [5]:
Z = Z_L + Z_R + Z_C
Z.expr

(L*s**2 + R*s + 1/C)/s

The current $I$ and the voltage $V$ across the circuit are related by:
$$
\frac{V}{I} = Z \Rightarrow \frac{I}{V} = Y \Rightarrow I = Y \cdot V
$$

The total admittance $Y$ of the circuit can be determined with:

In [6]:
Y = 1 / Z
Y.expr

s/(L*(s**2 + R*s/L + 1/(C*L)))

The voltage $V$ across the circuit can be defined as a *Sympy* symbol. Now, we can also determine the current $I$ and the voltage $V_c$ across the capacitor: 

In [7]:
V = sp.Symbol('V')
I = Y * V
V_C = Z_C * I
V_C.expr

V/(C*L*(s**2 + R*s/L + 1/(C*L)))

The transfer function we are looking for is $V_c / V$, with $V$ the input and $V_c$ the output of the system:

In [8]:
G = V_C / V
G.expr

1.0/(C*L*(s**2 + R*s/L + 1/(C*L)))

In [9]:
type(G)

python_control.core.transfer_function.TransferFunction

### Example 2.10 : Transfer Function - Multiple Loops

Given the network below, find the transfer function $I_2(s) / V(s)$.

![two-loop electrical network](./images/example_2-10.png)

To solve this problem we can use the classes `Mesh` and `Circuit` in module `mesh_analysis` of subpackage `python_control.electrical`.

First, we create the electrical components that are present in the multi-loop circuit. The voltage source is represented by a *Sympy* symbol.

In [10]:
V = sp.Symbol('V')
resistor1 = Resistor('R1')
inductor = Inductor('L')
resistor2 = Resistor('R2')
capacitor = Capacitor('C')

Next, we create each mesh (loop) and we add the components to each mesh that are part of that mesh:   

In [11]:
mesh1 = Mesh('1')
mesh1.add_voltage_source(V)
mesh1.add_component(resistor1)
mesh1.add_component(inductor)

mesh2 = Mesh('2')
mesh2.add_component(resistor2)
mesh2.add_component(capacitor)
mesh2.add_component(inductor)

Once the meshes are configured, we create the circuit and we add the meshes to this circuit:

In [12]:
circuit = Circuit()
circuit.add_mesh(mesh1)
circuit.add_mesh(mesh2)

Now that the circuit is fully configured, the circuit can be solved for the mesh currents: 

In [13]:
currents = circuit.solve()
currents

{I_1: (C*L*V*s**2 + C*R2*V*s + V)/(C*L*R1*s**2 + C*L*R2*s**2 + C*R1*R2*s + L*s + R1),
 I_2: C*L*V*s**2/(C*L*R1*s**2 + C*L*R2*s**2 + C*R1*R2*s + L*s + R1)}

We need the transfer function $G(s) = I_2(s) / V(s)$:

In [14]:
I_2 = currents[mesh2.current]
V = mesh1.voltage_sources[0]
G = I_2 / V
G

C*L*s**2/(C*L*R1*s**2 + C*L*R2*s**2 + C*R1*R2*s + L*s + R1)

Note that `G` is a *Sympy* expression. We can pass this expression to the `__init__` method of class `TransferFunction`:

In [15]:
type(G)

sympy.core.mul.Mul

In [16]:
G = TransferFunction(G)
G.expr

s**2/((R1 + R2)*(R1/(C*L*R1 + C*L*R2) + s**2 + s*(C*R1*R2 + L)/(C*L*R1 + C*L*R2)))

### Example 2.13 : Mesh Equations via Inspection

Write, but do not solve, the mesh equations for the network shown in the figure below.

![three-loop electrical network](./images/example_2-13.png)

Components:

In [17]:
V = sp.Symbol('V')
R1 = Resistor(Q_(1, 'ohm'))
R2 = Resistor(Q_(1, 'ohm'))
L1 = Inductor(Q_(2, 'H'))
L2 = Inductor(Q_(4, 'H'))
L3 = Inductor(Q_(3, 'H'))
C = Capacitor(Q_(1, 'F'))

Meshes and Circuit:

In [18]:
mesh1 = Mesh('1')
mesh1.add_voltage_source(V)
mesh1.add_component(R1)
mesh1.add_component(R2)
mesh1.add_component(L1)

mesh2 = Mesh('2')
mesh2.add_component(L2)
mesh2.add_component(L3)
mesh2.add_component(L1)
mesh2.add_component(R2)

mesh3 = Mesh('3')
mesh3.add_component(C)
mesh3.add_component(L2)
mesh3.add_component(R1)

circuit = Circuit()
circuit.add_mesh(mesh1)
circuit.add_mesh(mesh2)
circuit.add_mesh(mesh3)

Solve the circuit for the mesh currents:

In [19]:
currents = circuit.solve()
currents

{I_1: (20.0*V*s**3 + 13.0*V*s**2 + 10.0*V*s + V)/(24.0*s**4 + 30.0*s**3 + 17.0*s**2 + 16.0*s + 1.0),
 I_2: (8.0*V*s**3 + 10.0*V*s**2 + 3.0*V*s + V)/(24.0*s**4 + 30.0*s**3 + 17.0*s**2 + 16.0*s + 1.0),
 I_3: (8.0*V*s**3 + 13.0*V*s**2 + V*s)/(24.0*s**4 + 30.0*s**3 + 17.0*s**2 + 16.0*s + 1.0)}

Get the transfer function $I_3(s) / V(s)$:

In [20]:
I_3 = currents[mesh3.current]
V = mesh1.voltage_sources[0]
G = TransferFunction(I_3 / V)
G.expr

(0.333333333333333*s**3 + 0.541666666666667*s**2 + 0.0416666666666667*s)/(1.0*s**4 + 1.25*s**3 + 0.708333333333333*s**2 + 0.666666666666667*s + 0.0416666666666667)